In [1]:
import pandas as pd
import numpy as np

In [7]:
path = 'C:\\Users\\jenny\\Documents\\NTNU\\Master\\my_supplementary_data\\database\\'
faprotax_df = pd.read_csv(path + 'raw_data\\faprotax.txt', sep ='\t')
faprotax_df.head(5)

,FARPOTAX DATABASE,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,methanotrophy,elements:C,H; main_element:C; electron_donor:C; electron_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,# - - - - - - - - - - - - - - - - - - - - - - ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,*Archaea*ANME-2D*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,# DOI:10.1038/nature12375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,*Methanoperedens*nitroreducens*,NaN,NaN,NaN,NaN,NaN,# DOI:10.1038/nature12375,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,*Methylococcaceae*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,# DOI:10.1007/0-387-30745-1_15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# merge "unnamed" columns into one, 'extra_info'

cols = list(faprotax_df.columns)
cols = cols[1:len(cols)]
faprotax_df['extra_info'] = ''
faprotax_df = faprotax_df.fillna('')
for c in range(0,len(cols)):
    faprotax_df['extra_info'] = faprotax_df['extra_info'] + faprotax_df[str(cols[c])]
    faprotax_df = faprotax_df.drop(str(cols[c]), axis = 1)

# add metabolism tag to separate column
faprotax_df['metabolism'] = ''
for r in range(0,len(faprotax_df)):
    if 'elements' in faprotax_df['extra_info'][r]:
        curr_metab = faprotax_df['FARPOTAX DATABASE'][r]
    faprotax_df['metabolism'][r] = curr_metab

# remove empty, header, and separator lines
for r in range(0,len(faprotax_df)):
    if (faprotax_df['FARPOTAX DATABASE'][r].strip() == ''
    or '#' in faprotax_df['FARPOTAX DATABASE'][r]
    or faprotax_df['FARPOTAX DATABASE'][r] == faprotax_df['metabolism'][r]):
        faprotax_df = faprotax_df.drop(r, axis = 0)

faprotax_df = faprotax_df.reset_index()
faprotax_df = faprotax_df.drop('index', axis = 1)

In [10]:
# handle 'add_groups'

rows_to_drop = ['init']

# as long as there are 'add_group:...' rows present, this loop will add those entries and remove the 'add_group' ro
# first turn dropped 81 rows:
#  [130, 131, 137, 138, 139, 140, 141, 142, 226, 227, 288, 289, 818, 819, 820, 821, 895, 896, 931, 932, 1085, 1095, 
#1105, 1106, 1107, 1247, 1711, 1727, 1728, 1729, 2476, 2477, 2478, 4955, 4956, 4957, 4958, 4959, 4960, 5380, 5381,5384,
#5387, 5388, 5389, 5390, 5787, 5904, 5905, 5906, 6153, 6154, 6441, 7244, 7245, 7246, 7247, 7248, 7249, 7582, 7583, 7584,
#7590, 7591, 7592, 7675, 7766, 7767, 8374, 8375, 8376, 8377, 8378, 8379, 8380, 8381, 8382, 8383, 8384, 8385, 8386]

# second did 53:
#  [8362, 8363, 9428, 9714, 9862, 9872, 10124, 10205, 10927, 10928, 10929, 10930, 10931, 10932, 11601, 11602, 11603, 
#11604, 11605, 11606, 12398, 12399, 12889, 12890, 13380, 13381, 13382, 13406, 13487, 13648, 13649, 13650, 13792, 13940,
#13950, 14163, 14432, 14464, 14465, 14466, 14698, 14699, 14700, 14732, 14733, 14734, 14884, 17790, 17791, 17982, 18141,
#18142, 18143]

# third did 1:
#  [20406]

# fourth did 0 - i.e. rows_to_drop was empty at this point, thus loop ended
#  []

while len(rows_to_drop) != 0:
    rows_to_drop = list()

    # handle rows with 'add_group:...'
    for r in range(0,len(faprotax_df)):
        if 'add_group' in faprotax_df['FARPOTAX DATABASE'][r]:
            # the metabolic function of the group that will be added to the current metabolic function 
            add_metab = faprotax_df['FARPOTAX DATABASE'][r].split(':')[1]
            # current metabolic function, will be added to the specified group
            curr_metab = faprotax_df['metabolism'][r]
            for s in range(0,len(faprotax_df)):
                if faprotax_df['metabolism'][s] == add_metab:
                    # create and add new entry for all organisms with given metabolism, and the current metabolism
                    new_entry = (faprotax_df['FARPOTAX DATABASE'][s],np.nan,curr_metab)
                    faprotax_df = faprotax_df.append(pd.DataFrame(data=[new_entry],columns = ['FARPOTAX DATABASE', 'extra_info', 'metabolism']), ignore_index = True)
            # save index of 'add_group:' row that was just handled, for deletion later
            rows_to_drop.append(r)
            
    # remove resolved rows of 'add_group:...'
    for i in rows_to_drop:
        faprotax_df = faprotax_df.drop(index = i, axis = 0)
    
    # reset index and delete old one, to prepare for new iteration or the finished daraframe
    faprotax_df = faprotax_df.reset_index()
    faprotax_df = faprotax_df.drop('index', axis = 1)

In [11]:
# handle subtract_group

# removal from 22291 to 22276 rows 

rows_to_drop = ['init']
org_to_drop = ['init']

while len(rows_to_drop) != 0:
    rows_to_drop = list()
    prev = len(faprotax_df)
    for r in range(0,len(faprotax_df)):
        # find row specifying what to remove
        if 'subtract_group' in faprotax_df['FARPOTAX DATABASE'][r]:
            if r not in rows_to_drop: rows_to_drop.append(r)
            remove_metab = faprotax_df['FARPOTAX DATABASE'][r].split(':')[1]
            from_metab = faprotax_df['metabolism'][r]
            remove_group = list()
            # find names of organism to remove
            for s in range(0,len(faprotax_df)):
                if remove_metab == faprotax_df['metabolism'][s]:
                    remove_group.append(faprotax_df['FARPOTAX DATABASE'][s])
            # find index of organism to remove within the current metab
            for t in range(0,len(faprotax_df)):
                if ((faprotax_df['FARPOTAX DATABASE'][t] in remove_group) & (faprotax_df['metabolism'][t] == from_metab)):
                    if t not in rows_to_drop: rows_to_drop.append(t)

    # remove handled 'subtract_group' rows and the specifiec groups
    for i in rows_to_drop:
        faprotax_df = faprotax_df.drop(index = i, axis = 0)

    # reset index and delete old one, to prepare for new iteration or the finished daraframe
    faprotax_df = faprotax_df.reset_index()
    faprotax_df = faprotax_df.drop('index', axis = 1)

In [13]:
# merge rows with the same organism name
faprotax_df = faprotax_df.fillna('').groupby('FARPOTAX DATABASE', as_index = False).agg({'metabolism': '|'.join,'extra_info': '|'.join}).reset_index().drop('index', axis = 1)

In [15]:
# remove entries with lowest taxonomic level higher than 'species'
 # some known high level entries found by revieweing the entry names ('high_taxa')
 # general high-level endings: -ceae, -ales, -acteria, -eota, -utes, etc.
 # other: first and second word identical, second word all caps or containing numbers

high_taxa = ('chlorobi', 'chlorobia', 'bacteroidia', 'deferribacteres', 'thermotogae', 'verrucomicrobia')

high_tax_level = list()
faprotax_df['name'] = faprotax_df['FARPOTAX DATABASE'].str.replace(' ','*').str.split('*')

for r in range(0,len(faprotax_df)):
    faprotax_df['name'][r] = [i for i in faprotax_df['name'][r] if i != '']
    
for r in range(0,len(faprotax_df)):
    first_word = faprotax_df['name'][r][0]
    last_7_1st_word = first_word[-7:].lower()
    last_4_1st_word = first_word[-4:].lower()
    last_3_1st_word = first_word[-3:].lower()
    
    if len(faprotax_df['name'][r])>1:
        second_word = faprotax_df['name'][r][1]
        if len(second_word)<4: last_4_2nd_word = second_word
        else: last_4_2nd_word = second_word[-4:].lower()  
    else:
        second_word = ''
        last_4_2nd_word = ''
        
    # if only one word (e.g. genus or higher)
    if (len(faprotax_df['name'][r]) < 2): high_tax_level.append(r)  
    # if words contains typical endings for higher taxionomic levels than genus 
    elif last_7_1st_word == 'acteria': high_tax_level.append(r)
    elif ((last_4_1st_word == 'ceae') | (last_4_1st_word == 'ales') | (last_4_1st_word == 'eota') | (last_4_1st_word == 'utes') | (last_4_1st_word == 'etes')): high_tax_level.append(r)
    elif last_3_1st_word == 'rae': high_tax_level.append(r)
    elif ((last_4_2nd_word == 'ceae') | (last_4_2nd_word == 'ales') | (last_4_2nd_word == 'eota') | (last_4_2nd_word == 'utes') | (last_4_2nd_word == 'etes')): high_tax_level.append(r)
    # known high tax levels
    elif first_word in high_taxa: high_tax_level.append(r)
    # same first and second word (if no specific entry available)
    elif first_word.lower() == second_word.lower(): high_tax_level.append(r)
    # second word is all caps, often true if missing either genus or species name 
    elif second_word == second_word.upper(): high_tax_level.append(r)
    # numbers in second word
    elif any(i.isdigit() for i in second_word): high_tax_level.append(r)
    
# remove found entries stored in the list 'high_tax_level'
for i in high_tax_level:
    faprotax_df = faprotax_df.drop(index = i, axis = 0)

faprotax_df = faprotax_df.reset_index()
faprotax_df = faprotax_df.drop('index', axis = 1)

In [16]:
# make columns for taxonomic data

faprotax_df['genus'] = ''
faprotax_df['species'] = ''
faprotax_df['strain'] = ''
faprotax_df['speciesStrain'] = ''
faprotax_df['speciesStrainComp'] = ''

for r in range(0,len(faprotax_df)):
    genus = str(faprotax_df['name'][r][0]).capitalize()
    species = genus+' '+str(faprotax_df['name'][r][1])
    strain = ''
    for i in range(2,len(faprotax_df['name'][r])):
        strain += faprotax_df['name'][r][i]+' '
    strain = strain.strip()
    faprotax_df['genus'][r] = genus
    faprotax_df['species'][r] = species
    faprotax_df['strain'][r] = strain
    faprotax_df['speciesStrain'][r] = species+' '+strain
    faprotax_df['speciesStrainComp'][r] = (species.lower()+strain.lower()).strip().replace('subsp.','').replace(' subsp ', '').replace('sp.','').replace(' sp ','').replace('-','',5).replace(' ','',5).replace('!','',5).replace('\"','',5).replace('&','',5).replace('(','',5).replace(')','',5).replace('[','',5).replace(']','',5).replace(',','',5).replace('.','',5).replace('/','',5).replace(':','',5).replace('_','',5).replace('\#','',5).replace('*','',5).replace('?','',5).replace('+','',5).replace('=','',5).replace('°','',5)

In [17]:
# cleaning the reference column
faprotax_df['ref'] = faprotax_df['extra_info'].str.replace('#','',5).str.strip()
faprotax_df['ref'] = faprotax_df['ref'].str.replace(';',',5')
for r in range(0,len(faprotax_df)):
    faprotax_df['ref'][r] = faprotax_df['ref'][r].rstrip('|')

In [19]:
# saving
faprotax_df[['genus','species','strain','speciesStrain','speciesStrainComp','metabolism','ref']].to_csv(path + 'prepared_data\\faprotax.csv', sep =';', index = False)